# Lesson 4: Machine Learning with Modern Portfolio Theory (MPT)

## Intro

In this notebook, you'll learn how to use modern portfolio theory (MPT) and the Sharpe ratio to plot and find optimal stock portfolios. You'll also use machine learning to predict the best portfolios. Finally, you'll evaluate the performance of the ML-predicted portfolios.

First, let's load some code that we'll need to get started:

In [ ]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install scikit-learn


import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

lng_df = pd.read_csv('data/lng_mpt.csv')
spy_df = pd.read_csv('data/spy_mpt.csv')
smlv_df = pd.read_csv('data/smlv_mpt.csv')

lng_df.set_index('Date', inplace=True)
lng_df.index = pd.to_datetime(lng_df.index)
spy_df.set_index('Date', inplace=True)
spy_df.index = pd.to_datetime(spy_df.index)
smlv_df.set_index('Date', inplace=True)
smlv_df.index = pd.to_datetime(smlv_df.index)

## Join Stock DataFrames and Calculate Returns 
Our first step towards calculating modern portfolio theory (MPT) portfolios is to get daily and monthly returns. Eventually we're going to get the best portfolios of each month based on the Sharpe ratio. The easiest way to do this is to put all our stock prices into one DataFrame, then to resample them to the daily and monthly time frames. We need daily price changes to calculate volatility, which we will use as our measure of risk.

### Instructions
* Join together `lng_df`, `spy_df`, and `smlv_df` using `pd.concat()` into the `full_df` DataFrame
* Resample the `full_df` to Business Month Start (`'BMS'`) frequency.
* Get the daily percent change of `full_df` with `.pct_change()`.

In [ ]:
# Join 3 stock dataframes together
full_df = pd.concat(____, axis=1).dropna()

# Resample the full dataframe to monthly timeframe
monthly_df = full_df.resample(____).first()

# Calculate daily returns of stocks
returns_daily = ____

# Calculate monthly returns of the stocks
returns_monthly = monthly_df.pct_change().dropna()
print(returns_monthly.tail())

## Calculate Covariances for Volatility

In MPT, we quantify risk via volatility. The math for calculating portfolio volatility is complex, and it requires daily returns covariances. We'll now loop through each month in the `returns_monthly` DataFrame, and calculate the covariance of the daily returns.

With pandas datetime indices, we can access the month and year with `df.index.month` and `df.index.year`. We'll use this to create a mask for `returns_daily` that gives us the daily returns for the current month and year in the loop. We then use the mask to subset the DataFrame like this: `df[mask]`. This gets entries in the `returns_daily` DataFrame which are in the current month and year in each cycle of the loop. Finally, we'll use pandas' `.cov()` method to get the covariance of daily returns.

### Instructions
* Loop through the index of `returns_monthly`.
* Create a mask for `returns_daily` which uses the current month and year from `returns_monthly`, and matches this to the current month and year from `i` in the loop.
* Use the mask on `returns_daily` and calculate covariances using `.cov()`.

In [ ]:
# Daily covariance of stocks (for each monthly period)
covariances = {}
rtd_idx = returns_daily.index
for i in returns_monthly.____:    
    # Mask daily returns for each month and year, and calculate covariance
    mask = (rtd_idx.month == i.month) & (rtd_idx.____ == i.____)
    
    # Use the mask to get daily returns for the current month and year of monthy returns index
    covariances[i] = returns_daily[____].cov()

print(covariances[i])

## Calculate Portfolios
We'll now generate portfolios to find each month's best one. `numpy`'s `random.random()` generates random numbers from a uniform distribution, then we normalize them so they sum to 1 using the /= operator. We use these weights to calculate returns and volatility. Returns are sums of weights times individual returns. Volatility is more complex, and involves the covariances of the different stocks.

Finally we'll store the values in dictionaries for later use, with months' dates as keys.

In this case, we will only generate 10 portfolios for each date so the code will run faster, but in a real-world use-case you'd want to use more like 1000 to 5000 randomly-generated portfolios for a few stocks.-m pip install matplotlib

### Instructions
* Generate **3** random numbers for the weights using `np.random.random()`.
* Calculate returns by taking the dot product (`np.dot()`; multiplies element-by-element and sums up two arrays) of `weights` with the monthly returns for the current `date` in the loop.
* Use the `.setdefault()` method to add an empty list (`[]`) to the `portfolio_weights` dictionary for the current `date`, then append `weights` to the list.

In [ ]:
portfolio_returns, portfolio_volatility, portfolio_weights = {}, {}, {}

# Get portfolio performances at each month
for date in sorted(covariances.keys()):
    cov = covariances[date]
    for portfolio in range(10):
        weights = np.random.random(____)
        weights /= np.sum(weights) # /= divides weights by their sum to normalize
        returns = np.dot(____, returns_monthly.loc[____])
        volatility = np.sqrt(np.dot(weights.T, np.dot(cov, weights)))
        portfolio_returns.setdefault(date, []).append(returns)
        portfolio_volatility.setdefault(date, []).append(volatility)
        portfolio_weights.setdefault(date, ____).append(____)
        
print(portfolio_weights[date][0])

## Plot Efficient Frontier
We can finally plot the results of our MPT portfolios, which shows the "efficient frontier". This is a plot of the volatility vs the returns. This can help us visualize our risk-return possibilities for portfolios. The upper left boundary of the points is the best we can do (highest return for a given risk), and that is the efficient frontier.

To create this plot, we will use the latest date in our `covariances` dictionary which we created a few exercises ago. This has dates as keys, so we'll get the sorted keys using `sorted()` and `.keys()`, then get the last entry with Python indexing (`[-1]`). Lastly we'll use `matplotlib` to scatter variance vs returns and see the efficient frontier for the latest date in the data.

### Instructions
* Get the latest date from the `covariances` dictionary - remember the dates are the keys.
* Plot the volatility vs returns (`portfolio_returns`) for the latest date in a scatter plot, and set the `alpha` value for transparency to be `0.1`.

In [ ]:
# Get latest date of available data
date = sorted(covariances.____)[____]  

# Plot efficient frontier
# warning: this can take at least 10s for the plot to execute...
plt.scatter(x=portfolio_volatility[date], y=____,  alpha=____)
plt.xlabel('Volatility')
plt.ylabel('Returns')
plt.show()

## Get Best Sharpe Ratios
We need to find the "ideal" portfolios for each date so we can use them as targets for machine learning. We'll loop through each date in `portfolio_returns`, then loop through the portfolios we generated with `portfolio_returns[date]`. We'll then calculate the Sharpe ratio, which is the return divided by volatility (assuming a no-risk return of 0).

We use `enumerate()` to loop through the returns for the current date (`portfolio_returns[date]`) and keep track of the index with `i`. Then we use the current date and current index to get the volatility of each portfolio with `portfolio_volatility[date][i]`. Finally, we get the index of the best Sharpe ratio for each date using `np.argmax()`. We'll use this index to get the ideal portfolio weights soon.

### Instructions
* Using `enumerate()`, enumerate the `portfolio_returns` for each date in the loop.
* For the current `date` in the loop, append to the `sharpe_ratio` dictionary entry with the return (`ret`) divided by `portfolio_volatility` for the current date and current `i` in the loops.
* Set the value for the current date's `max_sharpe_idxs` to be the index of the maximum Sharpe ratio using `np.argmax()`.

In [ ]:
# Empty dictionaries for sharpe ratios and best sharpe indexes by date
sharpe_ratio, max_sharpe_idxs = {}, {}

# Loop through dates and get sharpe ratio for each portfolio
for date in portfolio_returns.keys():
    for i, ret in enumerate(portfolio_returns[____]):
    
        # Divide returns by the volatility for the date and index, i
        sharpe_ratio.setdefault(date, []).append(____ / portfolio_volatility[date][____])

    # Get the index of the best sharpe ratio for each date
    max_sharpe_idxs[date] = np.argmax(sharpe_ratio[____])

print(portfolio_returns[date][max_sharpe_idxs[date]])

## Calculate EWMAs
We will now work towards creating some features to be able to predict our ideal portfolios. We will simply use the price movement as a feature for now. To do this we will create a daily exponentially-weighted moving average (EWMA), then resample that to the monthly timeframe. Finally, we'll shift the monthly moving average of price one month in the future, so we can use it as a feature for predicting future portfolios.

### Instructions
* Use a `span` of 30 to calculate the daily exponentially-weighted moving average (`ewa_daily`).
* Resample the daily ewma to the month by using the Business Monthly Start frequency (BMS) and the first day of hte month (`.first()`).
* Shift `ewma_monthly` by one month forward, so we can use the previous month's EWMA as a feature to predict the next month's ideal portfolio.

In [ ]:
# Calculate exponentially-weighted moving average of daily returns
ewma_daily = returns_daily.ewm(span=____).mean()

# Resample daily returns to first business day of the month with the first day for that month
ewma_monthly = ewma_daily.resample(____).____

# Shift ewma for the month by 1 month forward so we can use it as a feature for future predictions 
ewma_monthly = ewma_monthly.____.dropna()

print(ewma_monthly.iloc[-1])

## Make Features and Targets
To use machine learning to pick the best portfolio, we need to generate features and targets. Our features were just created in the last exercise – the exponentially weighted moving averages of prices. Our targets will be the best portfolios we found from the highest Sharpe ratio.

We will use pandas' `.iterrows()` method to get the `index, value` pairs for the `ewma_monthly` DataFrame. We'll set the current value of ewma_monthly in the loop to be our features. Then we'll use the index of the best Sharpe ratio (from `max_sharpe_idxs`) to get the best `portfolio_weights` for each month and set that as a target.

### Instructions
* Use the `.iterrows()` method with `ewma_monthly` to iterate through the `index_value` in the loop.
* Use the `date` in the loop and `best_idx` to index `portfolio_weights` to get the ideal portfolio weights based on the best Sharpe ratio.
* Append the `ewma` to the features.

In [ ]:
targets, features = [], []

# Create features from price history and targets as ideal portfolio
for date, ewma in ewma_monthly.____:

    # Get the index of the best sharpe ratio
    best_idx = max_sharpe_idxs[date]
    targets.append(portfolio_weights[____][____])
    features.append(____)  # add ewma to features

targets = np.array(targets)
features = np.array(features)
print(targets[-5:])

## Plot Efficient Fronteir with Best Sharpe Ratio
Let's now plot the efficient frontier again, but add a marker for the portfolio with the best Sharpe index. Visualizing our data is always a good idea to better understand it.

Recall the efficient frontier is plotted in a scatter plot of portfolio volatility on the x-axis, and portfolio returns on the y-axis. We'll get the latest date we have in our data from `covariances.keys()`, although any of the `portfolio_returns`, etc, dictionaries could be used as well to get the date. Then we get volatilities and returns for the latest date we have from our `portfolio_volatility` and `portfolio_returns`. Finally we get the index of the portfolio with the best Sharpe index from `max_sharpe_idxs[date]`, and plot everything with `plt.scatter()`.

### Instructions
* Set `cur_volatility` to be the portfolio volatilities for the latest `date`.
* Construct the "efficient frontier" plot by plotting volatility on the x-axis and returns on the y-axis.
* Get the best portfolio index for the latest `date` from `max_sharpe_idxs`.

In [ ]:
# Get most recent (current) returns and volatility
date = sorted(covariances.keys())[-1]
cur_returns = portfolio_returns[date]
cur_volatility = ____

# Plot efficient frontier with sharpe as point
plt.scatter(x=____, y=____, alpha=0.1, color='blue')
best_idx = max_sharpe_idxs[____]

# Place an orange "X" on the point with the best Sharpe ratio
plt.scatter(x=cur_volatility[best_idx], y=cur_returns[best_idx], marker='x', color='orange')
plt.xlabel('Volatility')
plt.ylabel('Returns')
plt.show()

## Make Predictoins with a Random Forest
In order to fit a machine learning model to predict ideal portfolios, we need to create train and test sets for evaluating performance. We will do this as we did in previous chapters, where we take our `features` and `targets` arrays, and split them based on a `train_size` we set. Often the train size may be around 70-90% of our data.

We then fit our model (a random forest in this case) to the training data, and evaluate the R^2 scores on train and test using `.score()` from our model. In this case, the hyperparameters have been set for you, but usually you'd want to do a search with `ParameterGrid` like we did in previous notebooks.

### Instructions
* Set the `train_size` to be 85% of the full training set data using the `.shape` property of `features`.
* Create train and test targets from `targets` using Python indexing.
* Fit the random forest model to the `train_features` and `train_targets`.

In [ ]:
# Make train and test features
train_size = int(0.85 * ____)
train_features = features[:train_size]
test_features = features[train_size:]
train_targets = targets[____]
test_targets = targets[____]

# Fit the model and check scores on train and test
rfr = RandomForestRegressor(n_estimators=300, random_state=42)
rfr.fit(____, ____)
print(rfr.score(train_features, train_targets))
print(rfr.score(test_features, test_targets))

## Get Predictions and First Evaluation
Now that we have a trained random forest model (`rfr`), we want to use it to get predictions on the test set. We do this to evaluate our model's performance – at a basic level, is it doing as well or better than just buying the index, SPY?

We'll use the typical sklearn `.predict(features)` method, then multiply our monthly returns by our portfolio predictions. We sum these up with `np.sum()` since this will have 3 rows for each month. Then we plot both the monthly returns from our predictions, as well as SPY and compare the two.

### Instructions
* Use the `rfr` random forest model's `.predict()` method to make predictions on `train_features` and `test_features`.
* Multiply the test set portion of `returns_monthly` by `test_predictions` to get the returns of our test set predictions.
* Plot the test set `returns_monthly` for `'SPY'` (everything from `train_size` to the end of the data).

In [ ]:
# Get predictions from model on train and test
train_predictions = rfr.predict(train_features)
test_predictions = ____

# Calculate and plot returns from our RF predictions and the SPY returns
test_returns = np.sum(returns_monthly.iloc[train_size:] * ____, axis=1)
plt.plot(test_returns, label='algo')
plt.plot(returns_monthly['SPY'].iloc[____], label='SPY')
plt.legend()
plt.show()

## Evaluate Returns
Let's now see how our portfolio selection would perform as compared with just investing in the SPY. We'll do this to see if our predictions are promising, despite the low R2 value.

We will set a starting value for our investment of $ 1000, then loop through the returns from our predictions as well as from SPY. We'll use the monthly returns from our portfolio selection and SPY and apply them to our starting cash balance. From this we will get a month-by-month picture of how our investment is doing, and we can see how our predictions did overall vs the SPY. Next, we can plot our portfolio from our predictions and compare it to SPY.

### Instructions
* Set the first list entries of both `algo_cash` and `spy_cash` to the same amount (`cash`).
* Multiply the cash in our `test_returns` loop by `1 + r` in order to apply the returns to our `cash`.
* As with the `test_returns` loop, in the SPY performance loop, append `cash` to spy_cash after multiplying by `1 + r` to add the returns to `cash`.


In [ ]:
# Calculate the effect of our portfolio selection on a hypothetical $1k investment
cash = 1000
algo_cash, spy_cash = [cash], ____  # set equal starting cash amounts
for r in test_returns:
    cash *= 1 + r
    algo_cash.append(cash)

# Calculate performance for SPY
cash = 1000  # reset cash amount
for r in returns_monthly['SPY'].iloc[train_size:]:
    cash ____ ____
    ____

print('algo returns:', (algo_cash[-1] - algo_cash[0]) / algo_cash[0])
print('SPY returns:', (spy_cash[-1] - spy_cash[0]) / spy_cash[0])

## Plot Returns
Lastly, we'll plot the performance of our machine-learning-generated portfolio versus just holding the SPY. We can use this as an evaluation to see if our predictions are doing well or not.

Since we already have `algo_cash` and `spy_cash` created, all we need to do is provide them to `plt.plot()` to display. We'll also set the label for the datasets with legend in `plt.plot()`.

### Instructions
* Use `plt.plot()` to plot the `algo_cash` (with label `'algo'`) and `spy_cash` (with label `'SPY'`).
* Use `plt.legend()` to display the legend.

In [ ]:
# Plot the algo_cash and spy_cash to compare overall returns
plt.plot(____, ____)
plt.plot(spy_cash, label='SPY')
____  # show the legend
plt.show()